In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
#!/usr/bin/env python
# coding: utf-8
import os, sys
import torch
import torch.nn.functional as F
from collections import defaultdict
from tqdm import tqdm
from transformers import AutoConfig, AutoTokenizer, AutoModel

sys.path.append('../../src')
from models.dpr import VanillaDPR
from arguments import get_rerank_parser
from util.dataset import get_rerank_dataset
from util.util import MixedPrecisionManager, query_tokenizer, doc_tokenizer, hugface_models, set_seed, trec_eval, query_tokenizer_paragraph, document_tokenizer_paragraph

os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [4]:
def score_agg(score_matrix, query_reps, doc_reps, args):
    if args.score_agg == "colbert":
        max_input, max_indx = torch.max(scores, dim=1)
        return torch.mean(max_input).item()
    
    elif args.score_agg == "vector_rrf" :
        final_query_rep = torch.mean(query_reps, dim =0, keepdim=True)
        
        rank_matrix = torch.zeros(score_matrix.shape)
        _argsort = torch.argsort(score_matrix, descending=True)
        for row in range(len(_argsort)):
            for col in range(len(_argsort[row])):
                indx = _argsort[row][col]
                rank_matrix[row][indx] = col + 1
            
        rrf = torch.add(rank_matrix, 60)   
        rrf = rrf.pow(-1)
        rrf = torch.mean(rrf, dim=0, keepdim=True)
        rrf = rrf.to(args.device)
        final_doc_rep = rrf.mm(doc_reps)
        return F.cosine_similarity(final_query_rep, final_doc_rep).item()
    
    else:
        raise NotImplementedError

In [5]:
def get_datasets(args):
    query_dataset, rerank_document_dataset, run = get_rerank_dataset(args)  
    
    query_loader = torch.utils.data.DataLoader(query_dataset,
                                                   batch_size=args.batch_size,
                                                   drop_last=False,
                                                   shuffle=False)
    
    document_loader = torch.utils.data.DataLoader(rerank_document_dataset,
                                                      batch_size=args.batch_size, 
                                                      drop_last=False)
    
    return query_loader, document_loader, run

In [6]:
BASE_PATH='/work/snaseri_umass_edu/'
params = f"""--mode AUTO-HITL --seed 42 --model_name xlm-roberta-base --output_dir {BASE_PATH}/better_P3/experiments/BetterRerankerS1/runs --checkpoint {BASE_PATH}/multi_lang_dpr/checkpoints/msmarco/dpr_baseline_msmaroco_xlmr_multi_lang/ar_zh_ru/checkpoint_at_0.pth --query_maxlen 32 --doc_maxlen 180 --rerank_topK 1000 --collection_file {BASE_PATH}/better_P3/data/BETTER_PHASE3_COMBO_ARABIC_FARSI_RUSSIAN/combined-corpus.jl --task_file {BASE_PATH}/better_P3/data/BETTER_PHASE3_COMBO_ARABIC_FARSI_RUSSIAN/ir-auto-hitl-eval-tasks.json --test_qrels {BASE_PATH}/better_P3/data/BETTER_PHASE3_COMBO_ARABIC_FARSI_RUSSIAN/IR-relevance-assessments.qrels.GALAGO --test_run {BASE_PATH}/better_P3/data/BETTER_PHASE3_COMBO_ARABIC_FARSI_RUSSIAN/62e171cedebb2f0d2619f2f6.FINAL.out""".replace("\n"," ")

In [7]:
parser = get_rerank_parser()
args = parser.parse_args(params.split(" "))

In [8]:
args.score_agg = "colbert"
args.batch_size = 1
args.stride = 10
args.rrf_k = 60
set_seed(args.seed)
args.device = torch.cuda.current_device()
os.makedirs(args.output_dir, exist_ok=True)

In [9]:
_, tokenizer_class, model_class = hugface_models(args.model_name)
query_encoder = model_class.from_pretrained(args.model_name)
doc_encoder = model_class.from_pretrained(args.model_name)
tokenizer = AutoTokenizer.from_pretrained(args.model_name)
model = VanillaDPR(query_encoder, doc_encoder, args)
model = model.to(args.device)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are ini

tensor([0], device='cuda:0')


In [10]:
# load checkpoint
if args.checkpoint:
    state_dict = torch.load(os.path.join(args.checkpoint))["model_state_dict"]
    model.load_state_dict(state_dict)

In [11]:
query_loader, document_loader, baseline_run = get_datasets(args)  

loading run data: 100%|███████████████████████████████████████████████████████████████████████████████████| 54000/54000 [00:00<00:00, 996767.18it/s]


In [12]:
model.eval()
rerank_run = defaultdict(dict)
METRIC = 'map'

In [13]:
with torch.no_grad():
    qreps = {}
    for item in tqdm(query_loader, desc=f'encode rerank queries'):
        qid = item['qid'][0]  
        qtext = item['qtext'][0]
        q_chunck_ids, q_chunk_mask = query_tokenizer_paragraph(qtext, args, tokenizer)
        q_reps = model.query(q_chunck_ids, q_chunk_mask)
        qreps[qid] = q_reps.detach().cpu()

encode rerank queries:   0%|                                                                                                 | 0/54 [00:00<?, ?it/s]

17


encode rerank queries:  11%|█████████▉                                                                               | 6/54 [00:01<00:06,  7.43it/s]

17
15
22
18
16
21
22
21
23
24


encode rerank queries:  39%|██████████████████████████████████▏                                                     | 21/54 [00:01<00:01, 26.27it/s]

...

14
15
15
15
13
15
13
15
15
19
14
16


encode rerank queries:  72%|███████████████████████████████████████████████████████████████▌                        | 39/54 [00:01<00:00, 43.09it/s]

13
12
13
13
11
11
13
15
15
14
14
19


encode rerank queries: 100%|████████████████████████████████████████████████████████████████████████████████████████| 54/54 [00:01<00:00, 27.50it/s]

17
19
17
17
20
18
18
17
16


In [14]:
with torch.no_grad():
    dreps = {}
    for item in tqdm(document_loader, desc=f'encode rerank documents'):
        docid, doctext = item   
        docid = docid[0]
        doctext = doctext[0]
        d_chunk_ids, d_chunk_mask = document_tokenizer_paragraph(doctext, args, tokenizer)
        d_reps = model.doc(d_chunk_ids, d_chunk_mask)
        dreps[docid] = d_reps.detach().cpu()

encode rerank documents: 100%|██████████████████████████████████████████████████████████████████████████████████| 7859/7859 [02:37<00:00, 49.79it/s]


In [15]:
rerank_run = defaultdict(dict)

with torch.no_grad():
    for qid in tqdm(qreps):
        rank_list = baseline_run[qid]
        query_rep = qreps[qid].to(args.device)
        for docid in rank_list:
            document_rep = dreps[docid].to(args.device)
            scores = model.score(query_rep, document_rep)
            score = score_agg(model.score(query_rep, document_rep), query_rep, document_rep, args)            
            rerank_run[qid][docid] = score

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 54/54 [00:05<00:00,  9.66it/s]


In [16]:
runf = os.path.join(args.output_dir, f'AUTO-HITL-DPR-Reranked_info_task_title-task_stmt-task_narr-task_docs_segment_text-req_text-req_docs_segment_text.run')

with open(runf, 'wt') as runfile:
    for qid in tqdm(rerank_run):
        scores = list(sorted(rerank_run[qid].items(), key=lambda x: (x[1], x[0]), reverse=True))
        for i, (did, score) in enumerate(scores):
            runfile.write(f'{qid} 0 {did} {i+1} {score} run\n')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 54/54 [00:00<00:00, 424.83it/s]


In [17]:
baseline_list = {}
for qid in baseline_run:
    ranklist = list(sorted(baseline_run[qid].items(), key=lambda x: (x[1], x[0]), reverse=True))
    baseline_list.setdefault(qid, [])
    for retdoc in ranklist:
        baseline_list[qid].append(retdoc[0])

In [18]:
fusion_run = defaultdict(dict)
for qid in tqdm(rerank_run):
    rerank_list = list(sorted(rerank_run[qid].items(), key=lambda x: (x[1], x[0]), reverse=True))
    for i in range(len(rerank_list)):
        did = rerank_list[i][0]
        rerank_at = i + 1
        baseline_at = baseline_list[qid].index(did) + 1
        fused_score = (1/(args.rrf_k + rerank_at)) + (1/(args.rrf_k + baseline_at))
        fusion_run[qid][did] = fused_score

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 54/54 [00:00<00:00, 98.88it/s]


In [19]:
fusion_file = os.path.join(args.output_dir, f'Fusion_AUTO-HITL-DPR-Reranked_info_task_title-task_stmt-task_narr-task_docs_segment_text-req_text-req_docs_segment_text_Baseline_rrf-k_{args.rrf_k}.run')
with open(fusion_file, 'wt') as runfile:
    for qid in fusion_run:
        dummy_score = 100000
        fused_scores = list(sorted(fusion_run[qid].items(), key=lambda x: (x[1], x[0]), reverse=True))
        for i, (did, score) in enumerate(fused_scores):
            runfile.write(f'{qid} 0 {did} {i+1} {dummy_score} run\n')
            dummy_score -= 1

In [ ]:
args.output_dir

# RoundRobin Fusion of Arabic and Russian

In [ ]:
arabic_run = read_run("/work/snaseri_umass_edu/better_P3/data/BETTER_PHASE3_COMBO_ARABIC_FARSI_RUSSIAN/runfiles/62e171cedebb2f0d2619f2f6.arabic.Request.RERANKED.out")

In [ ]:
russian_run = read_run("/work/snaseri_umass_edu/better_P3/data/BETTER_PHASE3_COMBO_ARABIC_FARSI_RUSSIAN/runfiles/62e171cedebb2f0d2619f2f6.russian.Request.RERANKED.out")

In [ ]:
def roundrobin(*iterables):
    "roundrobin('ABC', 'D', 'EF') --> A D E B F C"
    # Recipe credited to George Sakkis
    pending = len(iterables)
    nexts = cycle(iter(it).next for it in iterables)
    while pending:
        try:
            for next in nexts:
                yield next()
        except StopIteration:
            pending -= 1
            nexts = cycle(islice(nexts, pending))

In [ ]:
round_robin_run = {}

for q in tqdm(arabic_run):
    ar_rank_list = list(sorted(arabic_run[q].items(), key=lambda x: x[1], reverse=True))
    ru_rank_list = list(sorted(russian_run[q].items(), key=lambda x: x[1], reverse=True))
    round_robin_run.setdefault(q,[])
    for item in zip(ar_rank_list, ru_rank_list):
        round_robin_run[q].append(item[0][0])
        round_robin_run[q].append(item[1][0])

In [ ]:
topk = 1000
round_robin_file= os.path.join(args.output_dir, f'round_robin_ar_ru_baseline.run')
with open(round_robin_file, 'wt') as runfile:
    for qid in round_robin_run:
        dummy_score = 100000
        rank = 1
        for did in round_robin_run[qid][:topk]:
            runfile.write(f'{qid} 0 {did} {rank} {dummy_score} run\n')
            dummy_score -= 1
            rank += 1